In [1]:
import os
import cv2
import matplotlib.pyplot as plt
from skimage.feature import local_binary_pattern
from skimage import feature
from skimage import data, exposure
import pandas as pd
import numpy as np
from tqdm import tqdm 
import gc

In [2]:
# settings for LBP
radius = 3
n_points = 8 * radius

# local_binary_pattern(x, n_points, radius)

In [ ]:
# HOG 

hog_vec, hog_vis = feature.hog((cv2.imread(filepath,0))
                               ,visualize=True, pixels_per_cell=(2,2)
                               , cells_per_block=(2,2),block_norm='L2')


plt.imshow(hog_vis)
print(hog_vec)
print(hog_vis.shape)


In [3]:
# norm func.
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()

def nor(x):
    
    return min_max_scaler.fit_transform(x)

In [ ]:
#  combine all 3 filter

k=np.stack([nor(data['X'][0]),nor(data['LBP'][0]),nor(data['HOG_vis'][0])],axis=2)
k.shape

In [ ]:
path='/home/raghav/Downloads/CK+48'
category=['fear', 'anger', 'surprise', 'happy', 'sadness', 'disgust', 'contempt']

# os.listdir('/home/raghav/Downloads/CK+48/fear')
# arr=cv2.imread
# arr
cv2.imread(path +'/'+category[0]+'/'+os.listdir(os.path.join(path,category[0]))[0]).shape

# plt.imread()


In [ ]:
# CK+ dataset building
x,y,lbp,hog_visual,hog_vector=[],[],[],[],[]
X1=[]
for i,label  in enumerate (category):
    for img in tqdm(os.listdir(os.path.join(path,label))):
        filepath= path + '/' + label + '/' + img
        arr = cv2.imread(filepath,0)
        LBP = local_binary_pattern(arr, n_points, radius)
        hog_vec, hog_vis = feature.hog(arr
                               ,visualize=True, pixels_per_cell=(2,2)
                               , cells_per_block=(2,2),block_norm='L2')
        x.append(arr)
        y.append(label)
        lbp.append(LBP)
        hog_visual.append(hog_vis)
        hog_vector.append(hog_vec)
#         X1.append(np.stack([nor(arr),nor(LBP),nor(hog_vis)],axis=2))
        
        
dat={'X':x,'LBP':lbp,'HOG_vis':hog_visual,'HOG_vec':hog_vector,'Y':y}
del x,lbp,hog_visual,hog_vector,y
data1=pd.DataFrame(dat)
del dat

In [ ]:
data1.to_pickle('/home/raghav/Downloads/data1.pkl')
del data1

In [4]:
# FER13 Data set

path='/home/raghav/Downloads/fer2013.csv'
emotion_label_to_text = {0:'anger', 1:'disgust', 2:'fear', 3:'happiness', 4: 'sadness', 5: 'surprise', 6: 'neutral'}

data=pd.read_csv(path)
p=len(data)//4
dat=[data.iloc[p:2*p],data.iloc[2*p:3*p],data.iloc[3*p:],]
del data

In [ ]:
def make_fer13(dat):
    
    itr=1
    for info in dat:
        x,y,lbp,hog_visual,hog_vector=[],[],[],[],[]
        X=[]
        
        y=(info.emotion)
        y=0
        itr=itr+1
        s='_'+str(itr)
        idx=0
        for i in tqdm(info.index):
            
            x.append( np.array(info.pixels[i].split(' ')).reshape(48, 48).astype('float32'))
            lbp.append(local_binary_pattern(x[idx], n_points, radius))
            hog_vec, hog_vis = feature.hog(x[idx],visualize=True, pixels_per_cell=(2,2), cells_per_block=(2,2),block_norm='L2')
            idx=idx+1
            hog_visual.append(hog_vis)
            hog_vector.append(hog_vec)
            
        
#         X.append(np.stack([nor(x[i]),nor(lbp[i]),nor(hog_visual[i])],axis=2))
        
        
        
        dat={'X':x,'LBP':lbp,'HOG_vis':hog_visual,'HOG_vec':hog_vector,'Y':y}
        #     print(dat['Y'].dtype)
        del x,lbp,hog_visual,hog_vector,y
        data2=pd.DataFrame(dat)
        del dat 
        data2.to_pickle('/home/raghav/Downloads/data2'+s+'.pkl')
        print("Checkpoint",itr,"made!  :  "+'/home/raghav/Downloads/data2'+s+'.pkl')
        print("Memory Flushed!")
        del data2
      
    
    return

make_fer13(dat)
    
# i = np.stack(i, axis=0)
# e = np.stack(enc,axis=0)
# imgs=np.hstack((i,e))
# img_array=imgs.reshape((21264,48,48,2))


  0%|          | 0/8971 [00:00<?, ?it/s]

Checkpoint 2 made!  :  /home/raghav/Downloads/data2_2.pkl
Memory Flushed!


100%|██████████| 8971/8971 [26:30<00:00,  5.64it/s]


Checkpoint 3 made!  :  /home/raghav/Downloads/data2_3.pkl
Memory Flushed!


 80%|███████▉  | 7176/8974 [20:54<05:45,  5.21it/s]

In [ ]:
# Concatenate Datasets

In [ ]:
path='/home/raghav/Downloads/data2_'
df=pd.read_pickle('/home/raghav/Downloads/data1.pkl')
for i in [1,3]:
    print("Pass: ",i)
#     if i+1 ==5:
#         break
    file=path + str(i) + '.pkl'
    print("Openinig File (",i,")")
    df_temp1=pd.read_pickle(file)
    print("Opened!")
    file=path + str(i+1) + '.pkl'
    print("Openinig File (",i+1,")")
    df_temp2=pd.read_pickle(file)
    print("Opened!")
    print("Combining!")
    df_temp1=pd.concat([df_temp1,df_temp2],ignore_index=True)
    print("Done!")
    del df_temp2
    df_temp1.to_pickle('/home/raghav/Downloads/df_'+str(i)+'.pkl')
    print("Checkpoint",i,"made!")
    print("Memory Flushed!")
    del df_temp1
    


# df1=pd.read_pickle('/home/raghav/Downloads/data2_1.pkl')
# df1.head()
# df2=pd.read_pickle('/home/raghav/Downloads/data2_2.pkl')
# df2.head()
# df3=pd.read_pickle('/home/raghav/Downloads/data2_3.pkl')
# df3.head()
# df4=pd.read_pickle('/home/raghav/Downloads/data2_4.pkl')
# df=pd.read_pickle('/home/raghav/Downloads/data1.pkl')

Pass:  1
Openinig File ( 1 )
Opened!
Openinig File ( 2 )


In [27]:
gc.collect()
# del df_temp2

20

In [28]:
# df_temp1=pd.concat([df_temp1,df_temp2],ignore_index=True)
# print("Done!")
# del df_temp2
# df_temp1.to_pickle('/home/raghav/Downloads/df_'+str(i)+'.pkl')
# print("Checkpoint",i,"made!")
# print("Memory Flushed!")
# del df_temp1
    